IMPORT GOOGLE DRIVE ΓΙΑ ΠΡΟΣΒΑΣΗ ΣΤΟΝ ΦΑΚΕΛΟ imagedb_btsd 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


UNZIP ΤΟΝ ΦΑΚΕΛΟ IMAGEDB_BTSD ΜΕΣΑ ΣΤΟΝ ΦΑΚΕΛΟ CONTENT ΓΙΑ ΑΜΕΣΗ ΠΡΟΣΠΕΛΑΣΗ




In [ ]:
import os
import zipfile

local_zip = '/content/drive/MyDrive/imagedb_btsd.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/ok')
zip_ref.close()

ΚΑΘΟΡΙΣΜΟΣ ΤΟΥ TRAIN DIRECTORY







In [ ]:
base_dir = '/content/ok'

train_dir = os.path.join(base_dir, 'imagedb')
print(train_dir)

/content/ok/imagedb


ΚΑΘΟΡΙΣΜΟΣ ΜΟΝΤΕΛΟΥ TRAINING

In [ ]:
import tensorflow as tf
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (5, 5), strides=(1, 1), activation='relu', input_shape=(28, 28, 1), padding = 'same'),
  tf.keras.layers.Conv2D(16, (5, 5), strides=(1, 1), activation='relu', input_shape=(28, 28, 1), padding = 'same'),
  tf.keras.layers.Conv2D(16, (5, 5), strides=(1, 1), activation='relu', input_shape=(28, 28, 1), padding = 'same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
  tf.keras.layers.Conv2D(16, (5, 5), strides=(1, 1), activation='relu', input_shape=(14, 14, 1), padding = 'same'),
  tf.keras.layers.Conv2D(16, (5, 5), strides=(1, 1), activation='relu', input_shape=(14, 14, 1), padding = 'same'),
  tf.keras.layers.Conv2D(16, (5, 5), strides=(1, 1), activation='relu', input_shape=(14, 14, 1), padding = 'same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
  tf.keras.layers.Conv2D(16, (5, 5), strides=(1, 1), activation='relu', input_shape=(7, 7, 1), padding = 'same'),
  tf.keras.layers.Conv2D(16, (5, 5), strides=(1, 1), activation='relu', input_shape=(7, 7, 1), padding = 'same'),
  tf.keras.layers.Conv2D(16, (5, 5), strides=(1, 1), activation='relu', input_shape=(7, 7, 1), padding = 'same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(34, activation='softmax') 
])

model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        416       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 16)        6416      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 16)        6416      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 16)        6416      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 16)        6416      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 16)        6

ΚΑΘΟΡΙΣΜΟΣ ΤΩΝ TRAIN ΚΑΙ VALIDATION DATAGENS ΜΕ ΟΡΙΣΜΟ ΔΙΑΦΟΡΩΝ AUGMENTETIONS ΣΤΟ TRAIN DATAGEN

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
shift = 0.03
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split=0.2,
                                   rotation_range=90,
                                   zoom_range=0.15,
                                   width_shift_range=shift,
                                   height_shift_range=shift
                                   )

val_datagen = ImageDataGenerator(rescale = 1./255,validation_split=0.2)

train_generator = train_datagen.flow_from_directory(train_dir,batch_size = 80,
                                                    class_mode = 'categorical',
                                                    target_size=(28,28),
                                                    color_mode='grayscale',
                                                    shuffle = True,
                                                    subset = 'training',
                                                    seed=4
                                                    
                                                    )
validation_generator = val_datagen.flow_from_directory(train_dir,
                                                    batch_size = 80,
                                                    class_mode = 'categorical',
                                                    color_mode='grayscale',
                                                    target_size=(28,28),
                                                    shuffle = True,
                                                    subset = 'validation',
                                                    seed=4
                                                    )


Found 2457 images belonging to 34 classes.
Found 599 images belonging to 34 classes.


OΡΙΣΜΟΣ CALLBACKS ΓΙΑ ΑΠΟΤΕΛΕΣΜΑΤΙΚΟΤΕΡΗ ΠΡΟΠΟΝΗΣΗ ΤΟΥ ΣΥΣΤΗΜΑΤΟΣ ΔΗΜΙΟΥΡΓΙΑ ΤΟΥ ΜΟΝΤΕΛΟΥ

In [ ]:
import datetime

callbacks = []
save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'best_weights.hdf5', save_best_only=True, verbose=1)
callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, verbose=1)
callbacks.append(early_stop_callback)

history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=30,
                              epochs=70,
                              validation_steps=5,
                              verbose=1,
                              callbacks=callbacks
                              )

Epoch 1/70
30/30 [==============================] - 9s 62ms/step - loss: 3.3856 - accuracy: 0.0698 - val_loss: 3.1779 - val_accuracy: 0.1100

Epoch 00001: val_loss improved from inf to 3.17795, saving model to best_weights.hdf5
Epoch 2/70
30/30 [==============================] - 1s 46ms/step - loss: 3.2042 - accuracy: 0.1334 - val_loss: 3.0371 - val_accuracy: 0.1850

Epoch 00002: val_loss improved from 3.17795 to 3.03706, saving model to best_weights.hdf5
Epoch 3/70
30/30 [==============================] - 1s 47ms/step - loss: 3.0520 - accuracy: 0.1898 - val_loss: 2.8612 - val_accuracy: 0.1675

Epoch 00003: val_loss improved from 3.03706 to 2.86116, saving model to best_weights.hdf5
Epoch 4/70
30/30 [==============================] - 1s 47ms/step - loss: 2.7987 - accuracy: 0.2287 - val_loss: 2.3274 - val_accuracy: 0.3875

Epoch 00004: val_loss improved from 2.86116 to 2.32744, saving model to best_weights.hdf5
Epoch 5/70
30/30 [==============================] - 1s 47ms/step - loss: 2.3

IMPORT TEST DATAGEN ΚΑΙ ΔΗΜΙΟΥΡΓΙΑ TEST DIRECTORY ΓΙΑ ΑΞΙΟΛΟΓΗΣΗ ΤΟΥ ΣΥΣΤΗΜΑΤΟΣ

In [ ]:
test_datagen  = ImageDataGenerator(rescale=1./255)
test_dir = os.path.join(base_dir, 'imagedb_test')
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=10,
                                                         class_mode  = 'categorical',
                                                         color_mode='grayscale',
                                                         target_size=(28,28)
                                                         ) 
loss, acc = model.evaluate(test_generator)

Found 2149 images belonging to 34 classes.
215/215 [==============================] - 1s 4ms/step - loss: 0.2462 - accuracy: 0.9372


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(28, 28), grayscale=True, interpolation='bilinear')
  plt.imshow(image.load_img(path), cmap='gray')
  plt.show()
  plt.imshow(img, cmap='gray')
  plt.show()

  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  
  images = np.vstack([x])
  
  model = tf.keras.models.load_model('/content/best_weights.hdf5')
  classes_pred = model.predict(images, batch_size=10)
  classes = ['apple', 'banana', 'strawberry']
  print(f'\n\n\n{classes}')
  print(f'\n\n\nSoftmax Output: {classes_pred}')
  print(f'\n\n\n{fn} is a(n) {classes[classes_pred.argmax()]}\n\n\n')
 